Хочется сделать максимально тупые бейзлайны без нейронок и без учёта контекста слов - чтобы понять, а сколько мы вообще выбили за счёт контекста. 

In [4]:
import numpy as np
import pandas as pd

In [5]:
import os

# Create labels for tagging

In [6]:
import os
import numpy as np
import pandas as pd
from ast import literal_eval
import re
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize

path = 'data/'

In [7]:
trial = pd.read_csv(path + 'tsd_trial.csv')
train = pd.read_csv(path + 'tsd_train.csv')
# final_test = pd.read_csv(path + 'tsd_test.csv')
final_test = pd.read_csv(path + 'tsd_test_gt.csv')

train['spans'] = train.spans.apply(literal_eval)
trial['spans'] = trial.spans.apply(literal_eval)
final_test['spans'] = final_test.spans.apply(literal_eval)
trial.shape, train.shape, final_test.shape

((690, 2), (7939, 2), (2000, 2))

In [8]:
print(len(set(trial.text).intersection(set(train.text))))
print(len(set(final_test.text).intersection(set(train.text))))

8
0


In [9]:
print((train.spans.apply(len) == 0).mean())
print((trial.spans.apply(len) == 0).mean())

0.06109081748331024
0.06231884057971015


In [10]:
import spans_utils
from importlib import reload
reload(spans_utils)
from spans_utils import display_spans, spans2labels, labels2spans

display_spans(trial.spans[0], trial.text[0])
display_spans(trial.spans[0], trial.text[0])

In [11]:
from tqdm.auto import tqdm, trange

In [12]:
import numpy as np
from semeval2021 import f1

# Словарь (naive bayes)

In [13]:
!source ~/p3/bin/activate; pip install pytextspan

You should consider upgrading via the '/home/dale/p3/bin/python -m pip install --upgrade pip' command.


In [14]:
import textspan

In [15]:
import nltk

In [16]:
from nltk import wordpunct_tokenize

In [17]:
text = train.text[0]
text

'Another violent and aggressive immigrant killing a innocent and intelligent US Citizen.... Sarcasm'

In [28]:
from tqdm.auto import tqdm, trange

In [48]:
from collections import Counter
toxic_vocab = Counter()
normal_vocab = Counter()

for i, row in tqdm(train.iterrows()):
    text = row.text.lower()
    toks = wordpunct_tokenize(text)
    spans = textspan.get_original_spans(toks, text)  # [[(0, 7)],[(8, 15)], ...]
    idxs = set(row.spans)
    for word, wordspan in zip(toks, spans):
        if idxs.intersection(set(range(*wordspan[0]))):
            toxic_vocab[word] += 1
        else:
            normal_vocab[word] += 1

In [49]:
len(toxic_vocab), len(normal_vocab)

(4516, 18613)

In [53]:
text = trial.text[0]
print(text)
print(trial.spans[0])

Because he's a moron and a bigot. It's not any more complicated than that.
[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]


In [77]:
alpha_pos = 0.1
alpha_neg = 1
threshold = 0.3


In [82]:
text = text.lower()
def get_toxic_spans(text, alpha_pos=0.1, alpha_neg=1, threshold=0.3):
    toks = wordpunct_tokenize(text)
    spans = textspan.get_original_spans(toks, text)
    result = []
    for word, wordspan in zip(toks, spans):
        score_pos = toxic_vocab.get(word, 0) + alpha_pos
        score_neg = normal_vocab.get(word, 0) + alpha_neg
        score = score_pos / (score_pos + score_neg)
        if score >= threshold:
            result.extend(range(*wordspan[0]))
    return result

get_toxic_spans(text)

[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]

In [85]:
for thr in [0.1, 0.3, 0.5, 0.7, 0.9]:
    preds = [get_toxic_spans(text, threshold=thr) for text in trial.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.1 0.48543603315613565
0.3 0.5611794534009353
0.5 0.5425581534714119
0.7 0.46653109132088505
0.9 0.1661587545991842


In [87]:
for thr in [0.2, 0.25, 0.3, 0.35, 0.4, 0.5]:
    preds = [get_toxic_spans(text, threshold=thr) for text in trial.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.2 0.5439401681820818
0.25 0.5526757386674521
0.3 0.5611794534009353
0.35 0.5589048654175035
0.4 0.5556682237445129
0.5 0.5425581534714119


In [90]:
for a in [0.01, 0.1, 0.3, 1, 3, 10]:
    preds = [get_toxic_spans(text, alpha_pos=a * 0.1, alpha_neg=a * 1) for text in trial.text]
    print(a, '\t',  np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.01 	 0.5540741406627043
0.1 	 0.5540741406627043
0.3 	 0.5540741406627043
1 	 0.5611794534009353
3 	 0.5607638742400142
10 	 0.554278235652248


Находка Игоря подтвердилась: тупой vocabulary-based подход даёт 56% точности. 

In [92]:
for thr in [0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, ]:
    preds = [get_toxic_spans(text, threshold=thr) for text in final_test.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, final_test.spans)]))

0.2 0.5267249460360243
0.25 0.5461798868031679
0.3 0.5689286860717168
0.35 0.579336301344286
0.4 0.5849523360901367
0.5 0.5902058076633944
0.6 0.5889286415675676
0.7 0.5585884191995162


# FastText

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
import numpy as np

In [93]:
from flair.embeddings import FastTextEmbeddings

# init embedding
embedding = FastTextEmbeddings('/home/dale/models/cc.en.300.bin')

In [94]:
ft = embedding.precomputed_word_embeddings.wv
ft

In [95]:
from functools import lru_cache

@lru_cache(maxsize=32_000)
def embed(w):
    return ft[w]

In [98]:
class FT(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return np.stack([embed(w) for w in X])

In [136]:
X = []
y = []
weights = []
for voc, label in [(toxic_vocab, 1), (normal_vocab, 0)]:
    for w, c in voc.items():
        weights.append(c)
        y.append(label)
        X.append(w)
np.random.seed(1)
perm = np.random.permutation(len(X))
X = np.array([X[i] for i in perm])
y = np.array([y[i] for i in perm])
weights = np.array([weights[i] for i in perm]) ** 1

In [142]:
pipe = make_pipeline(FT(), LogisticRegression(max_iter=1_000))
pipe.fit(X, y, logisticregression__sample_weight=weights)

Pipeline(steps=[('ft', FT()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [143]:
@lru_cache(maxsize=32_000)
def get_word_toxicity(w):
    return pipe.predict_proba([w])[0, 1]

In [158]:
def get_toxic_spans(text, threshold=0.3, model_weight=30.0):
    toks = wordpunct_tokenize(text)
    spans = textspan.get_original_spans(toks, text)
    result = []
    for word, wordspan in zip(toks, spans):
        mscore = get_word_toxicity(word)
        
        score_pos = toxic_vocab.get(word, 0) + alpha_pos + mscore * model_weight
        score_neg = normal_vocab.get(word, 0) + alpha_neg + (1-mscore) * model_weight
        
        score = (score_pos ) / (score_pos + score_neg)
        
        if score >= threshold:
            result.extend(range(*wordspan[0]))
    return result

* вес=корень - 50%
подбор весов примеров для обучения
* вес=корень счётчика - фигня
* вес=счётчик - 58%

подбор веса модели 
* вес модели - 1 (относительно словаря) - 60.38%
* вес модели - 3 - 62.17%
* вес модели - 10 - 62.31%
* вес модели - 30 - 61.64%

In [159]:
for thr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    preds = [get_toxic_spans(text, threshold=thr) for text in trial.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.1 0.4888441281425792
0.2 0.6009287884814687
0.3 0.5999739062936595
0.4 0.5826932918487115
0.5 0.5502984282946477
0.6 0.5087527525417305
0.7 0.41828741844146045
0.8 0.35338536914100604
0.9 0.16325601546562668


In [160]:
for thr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    preds = [get_toxic_spans(text, threshold=thr) for text in final_test.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, final_test.spans)]))

0.1 0.42374356540593644
0.2 0.5787110208319759
0.3 0.615733746342461
0.4 0.6110432484024746
0.5 0.6164045187730383
0.6 0.5925740101243221
0.7 0.5293337819949625
0.8 0.4616237834867994
0.9 0.2955879117231959


### Нейросетка

In [172]:
X = []
y = []
for voc, label in [(toxic_vocab, 1), (normal_vocab, 0)]:
    for w, c in voc.items():
        for _ in range(c):
            y.append(label)
            X.append(w)
np.random.seed(1)
perm = np.random.permutation(len(X))
X = np.array([X[i] for i in perm])
y = np.array([y[i] for i in perm])

In [173]:
len(X)

342965

In [229]:
from sklearn.neural_network import MLPClassifier
pipe = make_pipeline(FT(), MLPClassifier(hidden_layer_sizes=(300,), activation='relu', verbose=1, early_stopping=True, validation_fraction=0.05))
pipe.fit(X, y)

Iteration 1, loss = 0.22402599
Validation score: 0.937781
Iteration 2, loss = 0.21325462
Validation score: 0.939588
Iteration 3, loss = 0.21155067
Validation score: 0.938655
Iteration 4, loss = 0.21047662
Validation score: 0.938597
Iteration 5, loss = 0.20965603
Validation score: 0.940288
Iteration 6, loss = 0.20881203
Validation score: 0.939997
Iteration 7, loss = 0.20814020
Validation score: 0.938480
Iteration 8, loss = 0.20757158
Validation score: 0.939530
Iteration 9, loss = 0.20707349
Validation score: 0.939647
Iteration 10, loss = 0.20651087
Validation score: 0.939588
Iteration 11, loss = 0.20606586
Validation score: 0.939122
Iteration 12, loss = 0.20562595
Validation score: 0.940055
Iteration 13, loss = 0.20521489
Validation score: 0.938947
Iteration 14, loss = 0.20480584
Validation score: 0.939647
Iteration 15, loss = 0.20452820
Validation score: 0.939647
Iteration 16, loss = 0.20403018
Validation score: 0.939530
Validation score did not improve more than tol=0.000100 for 10 co

Pipeline(steps=[('ft', FT()),
                ('mlpclassifier',
                 MLPClassifier(early_stopping=True, hidden_layer_sizes=(300,),
                               validation_fraction=0.05, verbose=1))])

In [230]:
@lru_cache(maxsize=32_000)
def get_word_toxicity(w):
    return pipe.predict_proba([w])[0, 1]

In [231]:
def get_toxic_spans(text, threshold=0.3, model_weight=3.0):
    toks = wordpunct_tokenize(text)
    spans = textspan.get_original_spans(toks, text)
    result = []
    prev = False
    prev_end = 0
    for word, wordspan in zip(toks, spans):
        mscore = get_word_toxicity(word)
        
        score_pos = toxic_vocab.get(word, 0) + alpha_pos + mscore * model_weight
        score_neg = normal_vocab.get(word, 0) + alpha_neg + (1-mscore) * model_weight
        
        score = (score_pos ) / (score_pos + score_neg)
        
        toxic = score >= threshold
        if toxic:
            if prev:
                result.extend(range(prev_end, wordspan[0][0]))
            result.extend(range(*wordspan[0]))
        
        if len(word) > 0:
            prev = toxic
            prev_end = wordspan[0][1]
    return result

Подбор веса модели (против веса словаря):
* вес модели - 1 - 0.6322
* вес модели - 3 - 0.6398 (выбрал его)
* вес модели - 10 - 0.6378
* вес модели - 30 - 0.6360

Другие параметры:
* сделал токсичными пропуски между токсичными словами - 0.6397 (не помогает)
* увеличил размер скрытого слоя с 30 до 100 - и выбил уже 0.6454
* увеличил размер скрытого слоя до 300 - вроде особо пользы нет, 0.6443



In [232]:
for thr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    preds = [get_toxic_spans(text, threshold=thr) for text in trial.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.1 0.4957608668206724
0.2 0.6083819087224832
0.3 0.625489948993125
0.4 0.6166048481994797
0.5 0.5879459188133708
0.6 0.5457713234165323
0.7 0.48764345238387613
0.8 0.3575961534803883
0.9 0.1661587545991842


In [233]:
for thr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    preds = [get_toxic_spans(text, threshold=thr) for text in final_test.text]
    print(thr,  np.mean([f1(p, y) for p, y in zip(preds, final_test.spans)]))

0.1 0.42538572416947895
0.2 0.5775798901676763
0.3 0.6262081107598961
0.4 0.6443802662357849
0.5 0.6435602124499669
0.6 0.6261727952332393
0.7 0.5775737073659616
0.8 0.4772399665569934
0.9 0.3052372126999968


В общем, я выбил 64.5% на тесте, пользуясь ТОЛЬКО отдельно взятыми словами, и оценивая их ансамблем из простого счётчика и MLP над fasttext-эмбеддингом. 

ВООБЩЕ САМАЯ НАИЛУЧШАЯ модель во всей дорожке выбила 70.8%, наилучшая из наших - 68.6% (уже после дедлайна). 